In [114]:
from sympy import *
import numpy as np

D = 2

## Definiendo simbolos
t, r, theta, phi = symbols('t, r, \\theta, \\varphi')
dt, dr, dtheta, dphi = symbols('dt dr dtheta dphi')     
l0, l1, l2, l3 = symbols("l_0 l_1 l_2 l_3")
l, lamb, p, q, n, L = symbols("l lamb p q n L")

deldel = Function("DELDEL")
delf = Function("DELF")
box = Function("\\Box")

def getFuncName(i):
    if i==0:
        return alfa.name
    if i==1:
        return F.name
    if i==2:
        return G.name
    
    
if D == 1:
    ndmat = np.array([[[symbols("a") for _x in range(4)] for _w in range(4)] for _i in range(3)])
    alfa = Function("a")(t, r)
    F = Function("F")(t, r)
    G = Function("G")(t, r)
elif D == 2:
    ndmat = np.array([[[[symbols("a") for _x in range(4)] for _y in range(4)] for _w in range(4)] for _i in range(3)])
    alfa = Function("a")(t, theta, r)
    F = Function("F")(t, theta, r)
    G = Function("G")(t, theta, r)
elif D == 3:
    ndmat = np.array([[[[[symbols("a") for _x in range(4)] for _y in range(4)] for _z in range(4)] for _w in range(4)] for _i in range(3)])
    alfa = Function("a")(t, phi, theta, r)
    F = Function("F")(t, phi, theta, r)
    G = Function("G")(t, phi, theta, r)


symbolsString = ""
if D == 1:
    for _f in range(3):
        for _t in range(4):
            for _r in range(4):
                ndmat[_f][_t][_r] = symbols(getFuncName(_f)+str(_t)+str(_r))
                symbolsString += str(ndmat[_f][_t][_r])+" "
elif D == 2:
    for _f in range(3):
        for _t in range(4):
            for _theta in range(4):
                for _r in range(4):
                    ndmat[_f][_t][_theta][_r] = symbols(getFuncName(_f)+str(_t)+str(_theta)+str(_r))
                    symbolsString += str(ndmat[_f][_t][_theta][_r])+" "
elif D == 3:
    for _f in range(3):
        for _t in range(4):
            for _phi in range(4):
                for _theta in range(4):
                    for _r in range(4):
                        ndmat[_f][_t][_phi][_theta][_r] = symbols(getFuncName(_f)+str(_t)+str(_phi)+str(_theta)+str(_r))
                        symbolsString += str(ndmat[_f][_t][_phi][_theta][_r])+" "



In [116]:
ndmat[0][0][2][1] 

a021

In [117]:
def DELDEL(f, g):
    #canon = -f.diff(t)*g.diff(t) + 1/l1**2*f.diff(r)*g.diff(r) + 1/l1**2*f.diff(theta)*g.diff(theta) + 1/l2**2*f.diff(phi)*g.diff(phi)
    canon = -f.diff(t)*g.diff(t) + f.diff(r)*g.diff(r) + f.diff(theta)*g.diff(theta) + f.diff(phi)*g.diff(phi)
    return canon

def BOX(f):
    canon = -f.diff(t, 2) + (f.diff(r, 2) + f.diff(theta, 2) + f.diff(phi, 2))
    #canon = -f.diff(t, 2) + 1/l1**2*f.diff(r, 2) + 1/l1**2*f.diff(theta, 2) + 1/l2 **2*f.diff(phi, 2)
    return canon

def DELf(f, g):
    #canon = DELDEL(f, g) + f*BOX(g)
    canon = -(f*g.diff(t)).diff(t) + (f*g.diff(r)).diff(r)+ (f*g.diff(theta)).diff(theta) + (f*g.diff(phi)).diff(phi)
    #canon = -(f*g.diff(t)).diff(t) + 1/l1**2*(f*g.diff(r)).diff(r)+ 1/l1**2*(f*g.diff(theta)).diff(theta) + 1/l2**2*(f*g.diff(phi)).diff(phi)
    return canon


with evaluate(False):
    motion_alfa = box(alfa) + 2*(1 + 1/2*t)*(theta*(1-theta) + r*(1-r))
    
    motion_F = box(F)
    motion_G = box(G)
    
display(motion_alfa)
display(motion_F)
display(motion_G)

(2*(0.5*t + 1))*(\theta*(1 - \theta) + r*(1 - r)) + \Box(a(t, \theta, r))

\Box(F(t, \theta, r))

\Box(G(t, \theta, r))

In [118]:
def expandToDerivatives(func):
    p, q, s, b = map(Wild, 'pqsb')
    res = func.replace(deldel(p, q)*s, lambda p, q, s : DELDEL(p, q)*s)
    res = res.replace(delf(p, q)*s, lambda p, q, s : DELf(p, q)*s)
    return res.replace(box(p)*s, lambda p, s : BOX(p)*s)

In [119]:
res3a = (expandToDerivatives(motion_alfa))
display("Alfa done.")
res3f = (expandToDerivatives(motion_F))
display("F done.")
res3g = (expandToDerivatives(motion_G))
display("G done.")

'Alfa done.'

'F done.'

'G done.'

In [120]:
res3a

(2*(0.5*t + 1))*(\theta*(1 - \theta) + r*(1 - r)) + Derivative(a(t, \theta, r), (\theta, 2)) + Derivative(a(t, \theta, r), (r, 2)) - Derivative(a(t, \theta, r), (t, 2))

In [121]:
def getDelta(i):
    if i == t:
        return dt
    if r == i:
        return dr
    if theta == i:
        return dtheta
    if phi == i:
        return dphi

def getIndex(i):
    if t == i:
        return 0
    if phi == i:
        return 1
    if theta == i:
        return 2
    if r == i:
        return 3

def getFuncIndex(f):
    if f==F:
        return 1
    if f==G:
        return 2
    if f==alfa:
        return 0
    
def isThis(s, ix):
    if ix != None and s in ix.keys():
        return ix[s]
    else: 
        return 0
#http://www.iitg.ac.in/gtm/files/chap2.pdf
#https://www.uio.no/studier/emner/matnat/math/MAT-INF1100/h07/undervisningsmateriale/kap7.pdf
# 2: 1
# 1: 0
# 0: -1

def getDiscreteSymbol(ndmat, f, ix=None):
    if D==3:
        canon = ndmat[getFuncIndex(f)][1+isThis(t, ix)][1+isThis(phi, ix)][1+isThis(theta, ix)][1+isThis(r, ix)]
    elif D==2:
        canon = ndmat[getFuncIndex(f)][1+isThis(t, ix)][1+isThis(theta, ix)][1+isThis(r, ix)]
    elif D==1:
        canon = ndmat[getFuncIndex(f)][1+isThis(t, ix)][1+isThis(r, ix)]
    
    return canon
    
def FOD(f, i, wrt):
    if i==t:
        canon = (getDiscreteSymbol(ndmat, f, ix={i:0}) - getDiscreteSymbol(ndmat, f, ix={i:-1}))/(getDelta(i))
    else:
        canon = (getDiscreteSymbol(ndmat, f, ix={i:0}) - getDiscreteSymbol(ndmat, f, ix={i:-1}))/(getDelta(i))
    return canon

def SOD(f, i, j, wrt):
    if i==j:
        if f!=wrt and i==t:
            canon = (getDiscreteSymbol(ndmat, f, ix={i:0}) - 2*getDiscreteSymbol(ndmat, f, ix={i:-1}) + getDiscreteSymbol(ndmat, f, ix={i:-2}))/(getDelta(i)**2)
        else:
            canon = (getDiscreteSymbol(ndmat, f, ix={i:1}) - 2*getDiscreteSymbol(ndmat, f, ix={i:0}) + getDiscreteSymbol(ndmat, f, ix={i:-1}))/(getDelta(i)**2)
    else:
        if i != t and t != j:
            canon = (getDiscreteSymbol(ndmat, f, ix={i:1, j:1}) + getDiscreteSymbol(ndmat, f, ix={i:-1, j:-1}) - getDiscreteSymbol(ndmat, f, ix={i:1, j:-1}) - getDiscreteSymbol(ndmat, f, ix={i:-1, j:1}) )/(4*getDelta(i)*getDelta(j))
        #backward difference
        elif (j==t):
            canon = (1/getDelta(i))*(((getDiscreteSymbol(ndmat, f, ix={i:1, j:0})-getDiscreteSymbol(ndmat, f, ix={i:0, j:0}))/(getDelta(j))) - ((getDiscreteSymbol(ndmat, f, ix={i:1, j:-1}) - getDiscreteSymbol(ndmat, f, ix={i:0, j:-1}))/(getDelta(j))))
            
            #canon = (1/(2*getDelta(i)*getDelta(j)))*(getDiscreteSymbol(ndmat, f, ix={i:0, j:1})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:1})-getDiscreteSymbol(ndmat, f, ix={i:0, j:-1})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:-1}))
        elif (i == t):
            #canon = (1/(2*getDelta(i)*getDelta(j)))*(getDiscreteSymbol(ndmat, f, ix={i:1, j:0})-getDiscreteSymbol(ndmat, f, ix={i:1, j:-1})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:0})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:-1}))
            canon = (1/getDelta(i))*(((getDiscreteSymbol(ndmat, f, ix={i:0, j:1})-getDiscreteSymbol(ndmat, f, ix={i:0, j:0}))/(getDelta(j))) - ((getDiscreteSymbol(ndmat, f, ix={i:-1, j:1}) - getDiscreteSymbol(ndmat, f, ix={i:-1, j:0}))/(getDelta(j))))
            
        else:
            print("what")
    return canon

def discreti(arg, wrt):
    if len(arg.variables) == 1:
        return FOD(arg.expr, arg.variables[0], wrt)
    elif len(arg.variables) == 2:
        return SOD(arg.expr, arg.variables[0], arg.variables[1], wrt)
    else:
        return None

In [122]:
def discretizeVariables(func):
    res = func.replace(r, r*dr)
    res = res.replace(theta, theta*dtheta)
    res = res.replace(phi, phi*dphi)
    return res.replace(t, l*dt)


def discretizeDerivatives(func, wrt):
    res = func.replace(lambda arg: arg.is_Derivative, lambda arg: discreti(arg, wrt))
    res = res.replace(F, getDiscreteSymbol(ndmat, F))
    res = res.replace(G, getDiscreteSymbol(ndmat, G))
    return res.replace(alfa, getDiscreteSymbol(ndmat, alfa))

In [123]:
res7a = discretizeDerivatives(res3a, alfa)
res7a = discretizeVariables(res7a)

res7f = discretizeDerivatives(res3f, F)
res7f = discretizeVariables(res7f)

res7g = discretizeDerivatives(res3g, G)
res7g = discretizeVariables(res7g)

In [124]:
expra = ccode(res7a).replace("\\", "")
exprf = ccode(res7f).replace("\\", "")
exprg = ccode(res7g).replace("\\", "")

In [125]:
expra

'2*(0.5*dt*l + 1)*(theta*dtheta*(-theta*dtheta + 1) + dr*r*(-dr*r + 1)) + (a101 - 2*a111 + a121)/pow(dtheta, 2) - (a011 - 2*a111 + a211)/pow(dt, 2) + (a110 - 2*a111 + a112)/pow(dr, 2)'

In [126]:
def getSymbolsUsed(expr):
    syms = ""
    for _y in expr.free_symbols:
        syms += str(_y)+" "
    return syms[:-1]

In [127]:
symma = getSymbolsUsed(res7a)
print("Symbols for a: \n [", symma, "]")

symmf = getSymbolsUsed(res7f)
print("Symbols for F: \n [", symmf, "]")

symmg = getSymbolsUsed(res7g)
print("Symbols for G: \n [", symmg, "]")


Symbols for a: 
 [ a110 dtheta a011 l dr a112 a211 a121 dt a101 r \theta a111 ]
Symbols for F: 
 [ F111 dtheta F011 dr F112 F211 dt F110 F121 F101 ]
Symbols for G: 
 [ G211 dtheta G101 G110 dr G011 dt G111 G121 G112 ]


In [128]:
import sage as sg
from sage.all import *
var(symma.replace("\\", ""))
var(symbolsString.replace("\\", ""))
var("E4 E5 E6 r theta global_phi")

pi_4 = E4*sin(dr*r)*sin(dtheta*theta)*sin(dphi*global_phi) 
pi_5 = E5*sin(dr*r)*sin(dtheta*theta)*sin(dphi*global_phi) 
pi_6 = E6*sin(dr*r)*sin(dtheta*theta)*sin(dphi*global_phi) 

#dAdt = "0 + pi_4"
dAdt = "0.5*(dr*r*(1 - dr*r)*dtheta*theta*(1 - dtheta*theta))"
dFdt = "0 + pi_5"
dGdt = "p/L + pi_6"
def solveWithSage(expr, symbols, x):
    wata = eval(expr)
    resa = sg.symbolic.relation.solve(wata, x)
    return resa[0]

In [129]:
def getDerivative(eq, var):
    if type(eq) == int:
        return 0
    else:
        try:
            return eq.derivative(var)
        except:
            return 0

def solveForFirstState(expr, dfdt, func, hasMixed):
    if D==1:
        auxt = eval(func+"21") - 2*dfdt*dt
        firsta = expr.subs({eval(func+"01") : auxt})
    elif D==2:
        auxt = eval(func+"211") - 2*dfdt*dt
        firsta = expr.subs({eval(func+"011") : auxt})
    elif D==3:
        auxt = eval(func+"2111") - 2*dfdt*dt
        firsta = expr.subs({eval(func+"0111") : auxt})
    
    if hasMixed:
        auxr = eval(func+"1211")-eval(func+"1111")+eval(func+"0111")-dt*dr*getDerivative(dfdt, r)
        auxtheta = eval(func+"1121")-eval(func+"1111")+eval(func+"0111")-dt*dtheta*getDerivative(dfdt, theta)
        auxphi = eval(func+"1112")-eval(func+"1111")+eval(func+"0111")-dt*dphi*getDerivative(dfdt, phi)
        firsta = firsta.subs({eval(func+"0211") : auxr})
        firsta = firsta.subs({eval(func+"0121") : auxtheta})
        firsta = firsta.subs({eval(func+"0112") : auxphi})
        firsta = firsta.subs({eval(func+"0111") : auxt})
    firstaa = sg.symbolic.relation.solve(firsta, eval(func+"2111"))
    if D==1:
        firstaa = sg.symbolic.relation.solve(firsta, eval(func+"21"))
    elif D==2:
        firstaa = sg.symbolic.relation.solve(firsta, eval(func+"211"))
    elif D==3:
        firstaa = sg.symbolic.relation.solve(firsta, eval(func+"2111"))
    try:
        return firstaa[0]#.collect_common_factors().full_simplify().collect_common_factors()
    except:
        return firstaa

def replaceOtherFunctionsFirstState(expr, func1, func2, df1dt, df2dt):
    auxtf1 = eval(func1+"1111") - df1dt*dt
    auxtf1f1 = eval(func1+"1111") + df1dt*dt
    auxtf2 = eval(func2+"1111") - df2dt*dt
    auxtf2f2 = eval(func2+"1111") + df2dt*dt

    auxrf1 = eval(func1+"1211")-eval(func1+"1111")+eval(func1+"0111")-dt*dr*getDerivative(df1dt, r)
    auxthetaf1 = eval(func1+"1121")-eval(func1+"1111")+eval(func1+"0111")-dt*dtheta*getDerivative(df1dt, theta)
    auxphif1 = eval(func1+"1112")-eval(func1+"1111")+eval(func1+"0111")-dt*dphi*getDerivative(df1dt, phi)
    
    firsta = expr.subs({eval(func1+"0211") : auxrf1})
    firsta = firsta.subs({eval(func1+"0121") : auxthetaf1})
    firsta = firsta.subs({eval(func1+"0112") : auxphif1})
    
    auxrf2 = eval(func2+"1211")-eval(func2+"1111")+eval(func2+"0111")-dt*dr*getDerivative(df2dt, r)
    auxthetaf2 = eval(func2+"1121")-eval(func2+"1111")+eval(func2+"0111")-dt*dtheta*getDerivative(df2dt, theta)
    auxphif2 = eval(func2+"1112")-eval(func2+"1111")+eval(func2+"0111")-dt*dphi*getDerivative(df2dt, phi)
    
    firsta = firsta.subs({eval(func2+"0211") : auxrf2})
    firsta = firsta.subs({eval(func2+"0121") : auxthetaf2})
    firsta = firsta.subs({eval(func2+"0112") : auxphif2})
    
    firsta = firsta.subs({eval(func1+"0111") : auxtf1})
    firsta = firsta.subs({eval(func2+"0111") : auxtf2})
    
    firsta = firsta.subs({eval(func1+"2111") : auxtf1f1})
    firsta = firsta.subs({eval(func2+"2111") : auxtf2f2})
    return firsta

In [132]:
if D==3:
    ra = solveWithSage(expra, symma, a2111)
elif D==2:
    ra = solveWithSage(expra, symma, a211)
elif D==1:
    ra = solveWithSage(expra, symma, a21)
print(ra)

a211 == ((a101 - 2*a111 + a121)*dr^2*dt^2 - ((a011 - 2*a111)*dr^2 - (a110 - 2*a111 + a112)*dt^2)*dtheta^2 - (dr^4*dt^3*dtheta^2*l + 2*dr^4*dt^2*dtheta^2)*r^2 - (dr^2*dt^3*dtheta^4*l + 2*dr^2*dt^2*dtheta^4)*theta^2 + (dr^3*dt^3*dtheta^2*l + 2*dr^3*dt^2*dtheta^2)*r + (dr^2*dt^3*dtheta^3*l + 2*dr^2*dt^2*dtheta^3)*theta)/(dr^2*dtheta^2)


In [133]:
fa = solveForFirstState(ra, eval(dAdt), "a", True)
fa = replaceOtherFunctionsFirstState(fa, "F", "G", eval(dFdt), eval(dGdt))
fa

a211 == 1/2*((a101 - 2*a111 + a121)*dr^2*dt^2 + (2*a111*dr^2 + (a110 - 2*a111 + a112)*dt^2)*dtheta^2 - (dr^4*dt^3*dtheta^2*l + 2*dr^4*dt^2*dtheta^2)*r^2 + (dr^4*dt*dtheta^4*r^2 - dr^2*dt^3*dtheta^4*l - dr^3*dt*dtheta^4*r - 2*dr^2*dt^2*dtheta^4)*theta^2 + (dr^3*dt^3*dtheta^2*l + 2*dr^3*dt^2*dtheta^2)*r - (dr^4*dt*dtheta^3*r^2 - dr^2*dt^3*dtheta^3*l - dr^3*dt*dtheta^3*r - 2*dr^2*dt^2*dtheta^3)*theta)/(dr^2*dtheta^2)

In [136]:
if D==3:
    rf = solveWithSage(exprf, symmf, F2111)
elif D==2:
    rf = solveWithSage(exprf, symmf, F211)
elif D==1:
    rf = solveWithSage(exprf, symmf, F21)
print(rf)

F211 == ((F101 - 2*F111 + F121)*dr^2*dt^2 - ((F011 - 2*F111)*dr^2 - (F110 - 2*F111 + F112)*dt^2)*dtheta^2)/(dr^2*dtheta^2)


In [137]:
ff = solveForFirstState(rf, eval(dFdt), "F", True)
ff = replaceOtherFunctionsFirstState(ff, "a", "G", eval(dAdt), eval(dGdt))

ff

F211 == 1/2*(2*E5*dr^2*dt*dtheta^2*sin(dphi*global_phi)*sin(dr*r)*sin(dtheta*theta) + (F101 - 2*F111 + F121)*dr^2*dt^2 + (2*F111*dr^2 + (F110 - 2*F111 + F112)*dt^2)*dtheta^2)/(dr^2*dtheta^2)

In [138]:
if D==3:
    rg = solveWithSage(exprg, symmg, G2111)
elif D==2:
    rg = solveWithSage(exprg, symmg, G211)
elif D==1:
    rg = solveWithSage(exprg, symmg, G21)
print(rg)

G211 == ((G101 - 2*G111 + G121)*dr^2*dt^2 - ((G011 - 2*G111)*dr^2 - (G110 - 2*G111 + G112)*dt^2)*dtheta^2)/(dr^2*dtheta^2)


In [139]:
fg = solveForFirstState(rg, eval(dAdt), "G", True)
fg = replaceOtherFunctionsFirstState(fg, "a", "F", eval(dAdt), eval(dFdt))

fg

G211 == 1/2*((G101 - 2*G111 + G121)*dr^2*dt^2 + (2*G111*dr^2 + (G110 - 2*G111 + G112)*dt^2)*dtheta^2 + (dr^4*dt*dtheta^4*r^2 - dr^3*dt*dtheta^4*r)*theta^2 - (dr^4*dt*dtheta^3*r^2 - dr^3*dt*dtheta^3*r)*theta)/(dr^2*dtheta^2)

In [193]:
import re
def i2i(arg, index):
    if index == "0":
        return "("+arg+"-1)"
    if index == "1":
        return "("+arg+")"
    if index == "2":
        return "("+arg+"+1)"
    #if index == "3":
    #    return "("+arg+"+1)"
def i2iTime(arg, index):
    if index == "0":
        return "(tm1)"
    if index == "1":
        return "(t)"
    if index == "2":
        return "(tp1)"
    #if index == "3":
    #    return "(tp1)"    
def getCArray(ids):
    res = ""
    args = ["t", "threadIdx.z", "threadIdx.y", "threadIdx.x"]
    sizes = ["L", "O", "N", "M"]
    while len(ids)!=4:
        ids = ids[0]+'1'+ids[1:]
        
    for i, c in enumerate(ids):
        if args[i] == "t":
            res += i2iTime(args[i], c)+","
        else:
            res += i2i(args[i], c)+","
        #for j,s in enumerate(sizes[i+1:]):
            #res+="*"+s
        #res+="+"
    return res[:-1]


def getExpressionInC(expr):
    
    traduc = ccode(expr)
    if D==3:
        indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d\d\d', traduc)]
        for i in reversed(indexes):
            traduc = traduc[:i+1]+"[I("+ getCArray(traduc[i+1:i+5]) +")]"+ traduc[i+5:]
    elif D==2:
        indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d\d', traduc)]
        for i in reversed(indexes):
            traduc = traduc[:i+1]+"[I("+ getCArray(traduc[i+1:i+4]) +")]"+ traduc[i+4:]
    elif D==1:
        indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d', traduc)]
        for i in reversed(indexes):
            traduc = traduc[:i+1]+"[I("+ getCArray(traduc[i+1:i+3]) +")]"+ traduc[i+3:]
    traduc = traduc.replace("a[I(", "shmem[CI(0,")
    traduc = traduc.replace("F[I(", "shmem[CI(1,")
    traduc = traduc.replace("G[I(", "shmem[CI(2,")
    
    traduc = traduc.replace("threadIdx.x+1", "tidxp")
    traduc = traduc.replace("threadIdx.x-1", "tidxm")
    traduc = traduc.replace("threadIdx.x-2", "tidxmm")
    traduc = traduc.replace("threadIdx.y+1", "tidyp")
    traduc = traduc.replace("threadIdx.y-1", "tidym")
    traduc = traduc.replace("threadIdx.y-2", "tidymm")
    traduc = traduc.replace("threadIdx.z+1", "tidzp")
    traduc = traduc.replace("threadIdx.z-1", "tidzm")    
    traduc = traduc.replace("threadIdx.z-2", "tidzmm")
    
    return traduc

In [194]:
def nops(expr):
    count = 0
    string = ccode(expr._sympy_())
    count += string.count("*")
    count += string.count("^")
    count += string.count("-")
    count += string.count("+")
    count += string.count("/")
    return count

print(nops(ra))
print(nops(fa))

53
70


In [195]:
expr_a = getExpressionInC(ra._sympy_())
expr_fa = getExpressionInC(fa._sympy_())
expr_fa[:]

'shmem[CI(0,(tp1),(threadIdx.z),(threadIdx.y),(threadIdx.x))] == (1.0/2.0)*(pow(dr, 2)*pow(dt, 2)*(shmem[CI(0,(t),(threadIdx.z),(tidym),(threadIdx.x))] - 2*shmem[CI(0,(t),(threadIdx.z),(threadIdx.y),(threadIdx.x))] + shmem[CI(0,(t),(threadIdx.z),(tidyp),(threadIdx.x))]) + pow(dtheta, 2)*(2*shmem[CI(0,(t),(threadIdx.z),(threadIdx.y),(threadIdx.x))]*pow(dr, 2) + pow(dt, 2)*(shmem[CI(0,(t),(threadIdx.z),(threadIdx.y),(tidxm))] - 2*shmem[CI(0,(t),(threadIdx.z),(threadIdx.y),(threadIdx.x))] + shmem[CI(0,(t),(threadIdx.z),(threadIdx.y),(tidxp))])) - pow(r, 2)*(pow(dr, 4)*pow(dt, 3)*pow(dtheta, 2)*l + 2*pow(dr, 4)*pow(dt, 2)*pow(dtheta, 2)) + r*(pow(dr, 3)*pow(dt, 3)*pow(dtheta, 2)*l + 2*pow(dr, 3)*pow(dt, 2)*pow(dtheta, 2)) + pow(theta, 2)*(pow(dr, 4)*dt*pow(dtheta, 4)*pow(r, 2) - pow(dr, 3)*dt*pow(dtheta, 4)*r - pow(dr, 2)*pow(dt, 3)*pow(dtheta, 4)*l - 2*pow(dr, 2)*pow(dt, 2)*pow(dtheta, 4)) - theta*(pow(dr, 4)*dt*pow(dtheta, 3)*pow(r, 2) - pow(dr, 3)*dt*pow(dtheta, 3)*r - pow(dr, 2)*pow(dt

In [196]:
expr_f = getExpressionInC(rf._sympy_())
expr_ff = getExpressionInC(ff._sympy_())
expr_ff

'shmem[CI(1,(tp1),(threadIdx.z),(threadIdx.y),(threadIdx.x))] == (1.0/2.0)*(2*E5*pow(dr, 2)*dt*pow(dtheta, 2)*sin(dphi*global_phi)*sin(dr*r)*sin(dtheta*theta) + pow(dr, 2)*pow(dt, 2)*(shmem[CI(1,(t),(threadIdx.z),(tidym),(threadIdx.x))] - 2*shmem[CI(1,(t),(threadIdx.z),(threadIdx.y),(threadIdx.x))] + shmem[CI(1,(t),(threadIdx.z),(tidyp),(threadIdx.x))]) + pow(dtheta, 2)*(2*shmem[CI(1,(t),(threadIdx.z),(threadIdx.y),(threadIdx.x))]*pow(dr, 2) + pow(dt, 2)*(shmem[CI(1,(t),(threadIdx.z),(threadIdx.y),(tidxm))] - 2*shmem[CI(1,(t),(threadIdx.z),(threadIdx.y),(threadIdx.x))] + shmem[CI(1,(t),(threadIdx.z),(threadIdx.y),(tidxp))])))/(pow(dr, 2)*pow(dtheta, 2))'

In [197]:
expr_g = getExpressionInC(rg._sympy_())
expr_fg = getExpressionInC(fg._sympy_())
expr_fg[62:]

'= (1.0/2.0)*(pow(dr, 2)*pow(dt, 2)*(shmem[CI(2,(t),(threadIdx.z),(tidym),(threadIdx.x))] - 2*shmem[CI(2,(t),(threadIdx.z),(threadIdx.y),(threadIdx.x))] + shmem[CI(2,(t),(threadIdx.z),(tidyp),(threadIdx.x))]) + pow(dtheta, 2)*(2*shmem[CI(2,(t),(threadIdx.z),(threadIdx.y),(threadIdx.x))]*pow(dr, 2) + pow(dt, 2)*(shmem[CI(2,(t),(threadIdx.z),(threadIdx.y),(tidxm))] - 2*shmem[CI(2,(t),(threadIdx.z),(threadIdx.y),(threadIdx.x))] + shmem[CI(2,(t),(threadIdx.z),(threadIdx.y),(tidxp))])) + pow(theta, 2)*(pow(dr, 4)*dt*pow(dtheta, 4)*pow(r, 2) - pow(dr, 3)*dt*pow(dtheta, 4)*r) - theta*(pow(dr, 4)*dt*pow(dtheta, 3)*pow(r, 2) - pow(dr, 3)*dt*pow(dtheta, 3)*r))/(pow(dr, 2)*pow(dtheta, 2))'

In [198]:
def writeDefinitionGPU(filename, template, expr, arr):
    import os.path
    data = ""
    #check if header already exist
    if os.path.isfile(filename+".cu"):
        #in that case, just append new declaration
        fin = open(filename+".cu", "rt")
        data = fin.read()
        fin.close()
    else:
        data = "//### THIS IS A GENERATED FILE - DO NOT MODIFY ### \n \
#include \""+filename.split('/')[-1]+".cuh\"\n \
#include \"SharedMem.cuh\"\n"
    string = ''
    if arr == 'a':
        string = '0'
    elif arr == 'F':
        string = '1'
    else:
        string = '2'
    data = data + '\n' + template + """{
    int r = blockIdx.x*blockDim.x + threadIdx.x;
    int theta = blockIdx.y*blockDim.y + threadIdx.y;
    int phi = blockIdx.z*blockDim.z + threadIdx.z;
    int global_phi = phi + phi_offset;

    extern __shared__ REAL shmem[];
    if (r<M && theta<N && phi<O){
        fillSharedMemory(shmem, a, F, G, M, N, O, global_phi, r, theta, phi);
    }
    __syncthreads();
    if (r<M && theta<N && phi<O){
        int tidxp = threadIdx.x+1;
        int tidxm = threadIdx.x-1;
        
        int tidyp = threadIdx.y+1;
        int tidym = threadIdx.y-1;
        
        int tidzp = threadIdx.z+1;
        int tidzm = threadIdx.z-1;
        """ + "\n \
\t\tshmem[CI("+string+", tp1, threadIdx.z, threadIdx.y, threadIdx.x)] = " + expr + ";\n\t}\n \
    if (r<M && theta<N && phi<O){\
        copySharedMemoryToGlobal"+str(arr)+"(shmem, a, F, G, M, N, O, global_phi, r, theta, phi, tp1);\n\
    }\n\
}"

    
    fin = open(filename+".cu", "wt")
    fin.write(data)
    fin.close()

def writeDeclarationGPU(filename, template):
    import os.path
    data = ""
    #check if header already exist
    if os.path.isfile(filename+".cuh"):
        #in that case, just append new declaration
        fin = open(filename+".cuh", "rt")
        data = fin.read()
        fin.close()
    else:
        data = "//### THIS IS A GENERATED FILE - DO NOT MODIFY ### \n \
#pragma once\n \
#include <iostream>\n \
#include <unistd.h>\n \
#include <vector>\n \
#include <string>\n \
#include <fstream>\n \
#include <cmath>\n \
#include \"defines.h\"\n \
using namespace std;\n"
    
    data = data + '\n' + template + ";"
    
    fin = open(filename+".cuh", "wt")
    fin.write(data)
    fin.close()

def cleanOldFilesGPU(file):
    import os
    try:
        os.remove(file + ".cu")    
        os.remove(file + ".cuh")
    except:
        return "Already gone."

def writePairs(filename, funcname, expr):
    writeDeclarationGPU(filename, "__global__ void "+funcname+"(REAL *a, REAL *F, REAL *G, size_t l, size_t tp1, size_t t, size_t tm1, size_t tm2, size_t M, size_t N, size_t O, size_t phi_offset, REAL dt, REAL dr, REAL dtheta, REAL dphi, REAL l_1, REAL l_2, REAL lamb, int p, int q, int L)")
    writeDefinitionGPU( filename, "__global__ void "+funcname+"(REAL *a, REAL *F, REAL *G, size_t l, size_t tp1, size_t t, size_t tm1, size_t tm2, size_t M, size_t N, size_t O, size_t phi_offset, REAL dt, REAL dr, REAL dtheta, REAL dphi, REAL l_1, REAL l_2, REAL lamb, int p, int q, int L)", expr, funcname[-1])


def findClosingPar(expr):
    let = 1
    i = 1
    #print(expr)
    while let > 0 and i<len(expr):
        if expr[i] == '(':
            let += 1
        elif expr[i] == ')':
            let -= 1
        i+=1
    return i-1

def replaceAllPow(expr):
    indexes = [m.start() for m in re.finditer('pow', expr)]
    for i in indexes[::-1]:
        open = i+4
        close = findClosingPar(expr[open:])+open
        exponent = expr[open:close].split(',')[-1]
        
        commas = [m.start() for m in re.finditer(',', expr[open:close])]
        #print(expr[open:close])
        argument = expr[open:open+commas[-1]]
        
        rep = ''
        for j in range(int(exponent)):
            rep += argument+'*'
        rep = rep[:-1]
        expr = expr[:i] +'(' + rep + ')'+ expr[close+1:]
    return expr  
        #print(expr)
        #return
        #print(rep)

In [199]:
expr_fanp = replaceAllPow(expr_fa)
expr_ffnp = replaceAllPow(expr_ff)
expr_fgnp = replaceAllPow(expr_fg)

In [200]:

writePairs("../gpu/src/EquationAlfa", "computeFirsta", expr_fanp[63:])

writePairs("../gpu/src/EquationF", "computeFirstF", expr_ffnp[63:])

writePairs("../gpu/src/EquationG", "computeFirstG", expr_fgnp[63:])
